In [ ]:

import random
import time
import math
from collections import Counter
import re

In [ ]:
import copy
global N_ITERATIONS
N_ITERATIONS = 1000000

In [ ]:
class SushiShop:
  TICKERS = {"OTR":1,"OYJ":1,"TTR":1,"AJI":1,"TAI":1,"KAN":1,"UNI":1,"FUG":1,"HMT":2,"FOA":2,"ANG":2,"UNG":2,"AKA":2,"HOT":2,
           "IKU":2,"STK":2,"TBU":2,"MGR":3,"SAL":3,"IWS":3,"AJI":3,"SMS":3,"SNM":3,"KTO":3,"ENG":3,"KNP":3,"KHD":3,"HKG":3,
           "EBI":4,"IKA":4,"TKO":4,"TMG":4,"NGT":4,"KZN":4,"CAL":4,"KPA":5,"NTM":5,"TKM":5,"COR":5,"INZ":5}

  def __init__(self,arrangement):
    self.arrangement = arrangement
    self.count = len(self.arrangement)
    # print(self.count)
    # print(self.arrangement)

  def __str__(self):
    return " ".join([sushi.ticker for sushi in self.arrangement])

  def __len__(self):
    return len(self.arrangement)

  @classmethod
  def from_tickers(cls,tickers):
    """Given a string of tickers, construct the current roulette arrangement"""
    tickers = re.findall('.{3}', tickers)

    #Create a list of Sushi objects
    arrangement = []
    for ticker in tickers: arrangement.extend([ticker]*SushiShop.TICKERS[ticker])
    arrangement = [Sushi(arrangement[i],i) for i in range(len(arrangement))]
    return SushiShop(arrangement=arrangement)

class Sushi:
  def __init__(self,ticker,index):
    self.ticker = ticker
    self.index = index

  def __str__(self):
    return f"{self.ticker}{self.index}"


def get_random_sample(default_weights,count,shop,dishes=5):
  weights = default_weights
  hits = []
  for i in range(dishes):
    hit = random.choices(shop.arrangement,weights=weights,k=1)[0]
    hits.append(hit.ticker)
    # print(hit)
    next = (hit.index+1)

    while weights[next%count] == 0:
      next += 1

    weights[next % count] += weights[hit.index]
    weights[hit.index] = 0
    # print(weights)
    # print("============")
  hits.sort()
  return " ".join(hits)
def simulated_games(shop,iterations=N_ITERATIONS):
  count = shop.count
  default_weights = [1]*shop.count
  occurances = []
  for i in range(iterations):
    hit = get_random_sample(default_weights.copy(),count,shop)
    occurances.append(hit)
  cnt = Counter(occurances)
  return cnt.items()

In [ ]:
#global variables
global PROBABILITIES
global PAYOUT
global COST


In [ ]:
def sublist(ls1, ls2):
    '''
    >>> sublist([], [1,2,3])
    True
    >>> sublist([1,2,3,4], [2,5,3])
    True
    >>> sublist([1,2,3,4], [0,3,2])
    False
    >>> sublist([1,2,3,4], [1,2,5,6,7,8,5,76,4,3])
    False
    '''
    ls = [element for element in ls1 if element in ls2 and ls1.count(element) <= ls2.count(element)]
    return True if len(ls) == len(ls1) else False

def get_prob(combo,probabilities):
  # print(probabilities)
  # print(combo)
  matches = [(" ".join(tup[0]),tup[1]) for tup in probabilities if sublist(combo,tup[0])]
  return matches

def get_total_probs(combination,probabilities):

  combo_list = combination.split(" ")
  combo_list.sort()
  unique_dishes = set(combo_list)
  common_prob = get_prob(combo_list,probabilities)
  # print(common_prob)
  common_prob = common_prob[0][1]

  tups = []
  for dish in unique_dishes:
    new_combo = combo_list.copy()
    new_combo.remove(dish)
    new_combo.sort()
    p = get_prob(new_combo,probabilities)
    # print(f"Combo {new_combo} has probability {p}!")
    for prob in p:
      # print(prob)
      if prob not in tups: tups.append(prob)
  # print(tups)
  return (common_prob,sum([prob[1] for prob in tups]))


def EV(combination,current_probabilities,free_choice,payout,input):
  prob_without_free_choice = current_probabilities[0]
  prob_with_free_choice = current_probabilities[1]
  # print(f"Payout: {payout_rate}, input: {input} free_choice probability: {free_choice}, no free_choice: {prob_without_free_choice} free choice: {prob_with_free_choice}")
  EV = (prob_without_free_choice * (1-free_choice)*(payout)) + (prob_with_free_choice * free_choice * payout)
  # print(EV)
  return EV - input


def calculate_payout(payout_rate,input,free_choice,prob_no_free,prob_free):
  """Calculate payout at current payout rate"""

  return ((payout_rate+1)*input)/(prob_no_free - (prob_no_free*free_choice) + (free_choice*prob_free))

def continuous_EV(tickers,COST):

  global PROBABILITIES

  #data cleaning, capitalize, strip, and delete extra strings
  tickers = tickers.upper().strip().replace(" ","")
  shop = SushiShop.from_tickers(tickers)


  results = sorted(simulated_games(shop),key=lambda tup: tup[1],reverse=True)
  PROBABILITIES = [(ticker.split(" "),(probability/N_ITERATIONS)) for ticker,probability in results]


def get_EV_current():
  global PROBABILITIES
  global COST
  while True:
    combo = input("Input the combination here: ")
    payout = int(input("Input the payout here: "))
    print(f"The expected value of {combo} is between: {EV(combo,get_total_probs(combo,PROBABILITIES),0.8,payout,COST)} and {EV(combo,get_total_probs(combo,PROBABILITIES),0.95,payout,COST)}")

In [ ]:
import numpy as np

In [ ]:
def find_prob_with_free_choice(combo_str):
  #combo is a list [1,2,3,]
  # print("input recieved is",combo_str)
  combo = combo_str

  op1 = combo[1:]

  op2 = combo[2:]
  op2.extend(combo[0])
  op3 = combo[3:]
  op3.extend(combo[0:2])
  op4 = combo[:4]

  #sort
  np.sort(op2)
  np.sort(op3)
  np.sort(op4)


  total_prob = 0

  for combo,prob in PROBABILITIES:
    if combo == op1 or combo == op2 or combo == op3 or combo == op4:
      total_prob += prob

  return total_prob


In [ ]:
def calc_optimal_bets():
  ls = []
  free_choice = 0.8
  payout_rate = 2.65
  COST = 10
  for combo,prob in PROBABILITIES:

    combo_str = " ".join(combo)
    #total prob with free choice
    t = get_total_probs(combo_str,PROBABILITIES)
    ls.append((combo,EV(combo,t,free_choice,calculate_payout(payout_rate,COST,t[1],1-free_choice,free_choice),COST)))
  ls.sort(reverse=True,key=lambda x: x[1])
  print(ls)

In [ ]:
def continuous_EV(tickers,COST=20):

  global PROBABILITIES

  #data cleaning, capitalize, strip, and delete extra strings
  tickers = tickers.upper().strip().replace(" ","")
  shop = SushiShop.from_tickers(tickers)


  results = sorted(simulated_games(shop),key=lambda tup: tup[1],reverse=True)
  PROBABILITIES = [(ticker.split(" "),(probability/N_ITERATIONS)) for ticker,probability in results]
  # calc_optimal_bets()

In [ ]:
len(PROBABILITIES)

192

In [ ]:
#calibrate payouts
continuous_EV("OYJ HMT IKU HOT NGT KZN",10)
calc_optimal_bets()


[(['HMT', 'HOT', 'IKU', 'KZN', 'NGT'], 15.790233151344854), (['HOT', 'IKU', 'KZN', 'NGT', 'NGT'], 14.181127119665728), (['HMT', 'HOT', 'KZN', 'NGT', 'NGT'], 13.809859231804879), (['HOT', 'IKU', 'KZN', 'KZN', 'NGT'], 13.805005133189756), (['HMT', 'IKU', 'KZN', 'KZN', 'NGT'], 13.721430035352462), (['HMT', 'IKU', 'KZN', 'NGT', 'NGT'], 13.60621668407888), (['HMT', 'HOT', 'KZN', 'KZN', 'NGT'], 13.57276204660796), (['HOT', 'KZN', 'KZN', 'NGT', 'NGT'], 12.409763589848932), (['HMT', 'KZN', 'KZN', 'NGT', 'NGT'], 12.037951350194671), (['IKU', 'KZN', 'KZN', 'NGT', 'NGT'], 11.929425953506446), (['HMT', 'IKU', 'KZN', 'NGT', 'OYJ'], 11.728248819670814), (['HMT', 'HOT', 'KZN', 'NGT', 'OYJ'], 11.411360377883689), (['HOT', 'IKU', 'KZN', 'NGT', 'OYJ'], 11.228038258592488), (['HMT', 'KZN', 'KZN', 'NGT', 'OYJ'], 10.488654045643152), (['HOT', 'KZN', 'KZN', 'NGT', 'OYJ'], 10.021002423116318), (['HOT', 'KZN', 'NGT', 'NGT', 'OYJ'], 9.897320373187252), (['HMT', 'KZN', 'NGT', 'NGT', 'OYJ'], 9.844207833855542), 

In [ ]:
calculate_payout(2.65,10,0.8,*get_total_probs("TMG TMG TMG NTM NTM",PROBABILITIES))

272.2348767932074

In [ ]:
calc_optimal_bets()

[('ANG COR COR INZ INZ', 0.05518)]
37.409221200000005
[('AKA COR COR INZ INZ', 0.0496908)]
37.006125000000004
[('AKA ANG COR INZ INZ', 0.048886)]
37.138194600000006
[('AKA ANG COR COR INZ', 0.0464872)]
36.60048600000001
[('COR COR COR INZ INZ', 0.0402692)]
25.348198200000013
[('COR COR INZ INZ INZ', 0.0401276)]
25.367027400000012
[('ANG COR INZ INZ INZ', 0.0364)]
25.996557600000013
[('AKA COR COR INZ TAI', 0.0321716)]
27.861576600000006
[('ANG COR COR COR INZ', 0.031942)]
25.133994600000012
[('AKA COR INZ INZ INZ', 0.0305736)]
25.009436400000002
[('AKA COR COR COR INZ', 0.0305048)]
25.29204600000001
[('COR COR INZ INZ TAI', 0.0302924)]
28.878681000000018
[('AKA COR INZ INZ TAI', 0.028008)]
26.90023440000001
[('AKA ANG COR INZ TAI', 0.026216)]
26.773593600000012
[('ANG COR COR INZ TAI', 0.0252436)]
27.17405340000001
[('ANG COR INZ INZ TAI', 0.0240164)]
27.0609846
[('COR COR COR INZ TAI', 0.021006)]
19.964029800000002
[('ANG ANG COR INZ INZ', 0.0200996)]
19.995089400000005
[('AKA ANG ANG

In [ ]:
calculate_payout(0.1,20,0.80,*get_total_probs("OYJ STK STK CAL CAL",PROBABILITIES))

[('CAL CAL OYJ STK STK', 0.0009712)]


630.9517390750708

In [ ]:
calculate_payout(0.2,10,0.85,*get_total_probs("ANG EBI EBI INZ INZ",probabilities))

48.74217764208498

In [ ]:
calculate_payout(0.2,10,0.85,*get_total_probs("INZ INZ INZ EBI EBI",probabilities))

74.6083759715021